In [1]:
%matplotlib notebook
# Continue to examine affect of ip address risk on passing rate.
# First, focus on period before Oct 27 when it was [72-80%]
# Second, show other periods
# Third, find the outliers for each week
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
import sys
from scipy.stats.mstats import winsorize
import piso                       # Provide methods for intervals
import mywhois

import dmv_test_input as dti      # Local l

In [2]:
# Intitalize data and parameters

def read_data():
    # Read and cleanse data from akts log and risk database
    df, risk = dti.dmv_risk_input(case=2, save=False)
    
    # Time in minutes. 
    # elapsed time from start to end of test. Sometimes they are out of order.
    df["elapsed"] = abs(df['TestEndDateTime'] - df['TestStartDateTime']).dt.total_seconds()/60.   
    df["passed"]  = (df.Result=="P")
    
    # Drop negative duration since they must be in error
    # Drop long durations because the chance of error is high
    df = df[ (df.duration>0) & (df.duration<120) & (df.elapsed<120)].reset_index()
       
    return df, risk

df, risk = read_data()

# Drop rows in akts where ip has undefined risk<0. They are probably 10. or 192.. nets
df.drop(df[df.score < 0].index, inplace=True)

CHANGEPT = 14.5                   # The passing rate changes at the changept
RATE = 0.67                       # Passing rate reaches steady value
SBIN, EBIN, INC = (5, 100, 5)     # Define bins for risk score
EPSILON = sys.float_info.epsilon  # Smallest float


Original length of sample data is 179142
2862 tests with Result, IPAddress, TotalScore = NaN dropped
Extra ip address dropped in 1183 tests


Risk.find: Could not find IPv4Address for ip_string='2001'
Risk.find: Could not find IPv4Address for ip_string='2001'
Risk.find: Could not find IPv4Address for ip_string='2001'
Risk.find: Could not find IPv4Address for ip_string='2001'
Risk.find: Could not find IPv4Address for ip_string='2001'
Risk.find: Could not find IPv4Address for ip_string='2001'


In [3]:
# # Here's what the risk object looks like
# for i, k in enumerate(risk.risk):
#     print(f'CIDR--> {k}')
#     if i>0: break
#     for j, k1 in enumerate(risk.risk[k]):
#         if k1=='risk_comment': continue
#         print(f'{k1:<12} {risk.risk[k][k1]}')
#     print()



In [4]:
def risk_toframe(risk):
    # Convert risk dict to dataframe
    tlist =[]
    for cidr in risk.risk:
        tmp = risk.risk[cidr]
        tmp['cidr'] = cidr
        tlist.append(tmp)
    return pd.DataFrame(tlist)

original = risk_toframe(risk)

In [5]:
tmp = original.copy()
tmp.head()
tmp = tmp.drop(columns=['risk_comment', 'timestamp', 'address'])

In [13]:
while True:
    ip = input('ip? ')
    if len(ip)==0:
        break
    t = risk.find(ip)
    if t is None:
        print('Not found')
    else:
        display(tmp[tmp.ip==ip])
print('Done!')              

        

ip? 216.151.180.112


,ip,score,risk,postalcode,state,country,countrycode,organization,city,handle,cidr


ip? 
Done!


In [14]:
tmp[tmp.ip=='216.151.180.112']

,ip,score,risk,postalcode,state,country,countrycode,organization,city,handle,cidr


In [15]:
risk.find('216.151.180.112')


{'ip': '216.151.180.64',
 'score': '100',
 'risk': 'very high',
 'risk_comment': 'We consider web traffic from IP address 216.151.180.64 to present a potentially very high fraud risk. This IP address is operated by Netprotect whose web traffic we consider to present a potentially high fraud risk, and is owned by Highwinds Network Group, Inc. whose web traffic we also consider to present a potentially high fraud risk. In each of these cases, non-web traffic may present a different risk or no risk at all. Scamalytics see medium levels of web traffic from this IP address across our global network, almost all of which we suspect to be potentially fraudulent. Considering only the web traffic where we have visibility, we apply a risk score of 100 /100 to 216.151.180.64 , which reflects the proportion of this traffic which we suspect to be potentially fraudulent. If you see web traffic from this IP address there is potentially a very high risk that it is criminals engaged in fraudulent activi

In [41]:
def get(ip):
#     import pdb; pdb.set_trace()
    t = mywhois.get_arin(ip)
    return t

while True:
    ip = input('ip? ')
    if len(ip)==0:
        break
    t = get(ip)
    if t is None:
        print('Not found')
    else:
        t[list(t.keys())[0]].pop('risk_comment')
#         display(t[list(t.keys())[0]])
        display(t)
print('Done!')              



ip? 216.73.160.176


{'216.73.160.0/23': {'ip': '216.73.160.176',
  'score': '100',
  'risk': 'very high',
  'address': ['60 Hudson St'],
  'postalcode': '10013',
  'state': 'NY',
  'country': 'United States',
  'countrycode': 'US',
  'organization': 'Bandito Networks',
  'city': 'New York',
  'handle': 'C08103672',
  'timestamp': '03:17:2022 14:14:13'}}

ip? 
Done!


In [10]:
a={'a':5, 'b':6}


In [25]:
print(a.pop('b'))

6


In [29]:
t[list(t.keys())[0]].pop['risk_comment']




TypeError: 'builtin_function_or_method' object is not subscriptable

In [35]:
t[list(t.keys())[0]].pop('risk_comment')


"IP address 192.142.128.165 is operated by LogicWeb Inc. whose web traffic we consider to present a potentially high fraud risk. This IP address is owned by M247 Ltd whose web traffic we also consider to present a potentially high fraud risk. In both cases, non-web traffic may present a different risk or no risk at all. Scamalytics see medium levels of traffic from LogicWeb Inc. across our global network, some of which we suspect to be potentially fraudulent. We have no visibility into the web traffic directly from 192.142.128.165 , and therefore apply a risk score of 63 /100 based on the overall risk from LogicWeb Inc. 's IP addresses where we do have visibility. If you see web traffic from this IP address there is potentially a high risk that it is criminals engaged in fraudulent activity. Other types of traffic may present a different risk or no risk. 192.142.128.165 is not a standard domestic connection, it is a commercial server which could be proxying traffic from another geograp

In [5]:
x=[1,2,3]
for t in x:
    if t==3:
        if not 4 in x:
#             x.append(4)
            x.insert(0,4)
    print(t)


1
2
3
3


In [6]:
x


[4, 1, 2, 3]